In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('ogs20-21data.txt')
winter2 = pd.read_csv ('ogs21-22data.txt')
winter3 = pd.read_csv ('ogs22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
0,OGS,2021-12-01 00:15:00,0.00,-SN
1,OGS,2021-12-01 00:35:00,0.00,-SN
2,OGS,2021-12-01 00:55:00,0.51,-SN
3,OGS,2021-12-01 01:15:00,0.00,-SN
4,OGS,2021-12-01 01:35:00,0.00,-SN
...,...,...,...,...
6354,OGS,2022-02-27 18:15:00,0.00,-SN
6355,OGS,2022-02-27 18:35:00,0.00,-SN
6356,OGS,2022-02-27 18:55:00,0.25,-SN
6357,OGS,2022-02-27 19:15:00,0.00,-SN


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 55]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 55]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 55]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
ogs = LocationInfo(
    'Ogdensburg', 'New York', 
    latitude=44.6942, 
    longitude=-75.4863, 
    timezone='America/New_York'
)

ogs

LocationInfo(name='Ogdensburg', region='New York', timezone='America/New_York', latitude=44.6942, longitude=-75.4863)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(ogs.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(ogs.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(ogs.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:19:46  2020-12-01 16:22:24  2020-12-01
1   2020-12-02 07:20:53  2020-12-02 16:22:05  2020-12-02
2   2020-12-03 07:21:58  2020-12-03 16:21:48  2020-12-03
3   2020-12-04 07:23:02  2020-12-04 16:21:34  2020-12-04
4   2020-12-05 07:24:05  2020-12-05 16:21:22  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:47:45  2021-02-24 17:43:04  2021-02-24
86  2021-02-25 06:46:05  2021-02-25 17:44:25  2021-02-25
87  2021-02-26 06:44:23  2021-02-26 17:45:47  2021-02-26
88  2021-02-27 06:42:41  2021-02-27 17:47:08  2021-02-27
89  2021-02-28 06:40:58  2021-02-28 17:48:28  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:19:30  2021-12-01 16:22:29  2021-12-01
1   2021-12-02 07:20:37  2021-12-02 16:22:09  2021-12-02
2   2021-12-03 07:21:43  2021-12-03 16:21:51  2021-12-03
3   2021-12-04 07:22:47  2021-12-04 16:21:37  2021-12-04
4   2021-12-05 07:23:50  2021-12-05 16:21:25  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:48:09  2022-02-24 17:42:44  2022-02-24
86  2022-02-25 06:46:29  2022-02-25 17:44:06  2022-02-25
87  2022-02-26 06:44:48  2022-02-26 17:45:27  2022-02-26
88  2022-02-27 06:43:06  2022-02-27 17:46:48  2022-02-27
89  2022-02-28 06:41:23  2022-02-28 17:48:09  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:19:14  2022-12-01 16:22:34  2022-12-01
1   2022-12-02 07:20:21  2022-12-02 16:22:13  2022-12-02
2   2022-12-03 07:21:27  2022-12-03 16:21:55  2022-12-03
3   2022-12-04 07:22:32  2022-12-04 16:21:40  2022-12-04
4   2022-12-05 07:23:35  2022-12-05 16:21:27  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:48:34  2023-02-24 17:42:24  2023-02-24
86  2023-02-25 06:46:53  2023-02-25 17:43:46  2023-02-25
87  2023-02-26 06:45:12  2023-02-26 17:45:07  2023-02-26
88  2023-02-27 06:43:31  2023-02-27 17:46:28  2023-02-27
89  2023-02-28 06:41:48  2023-02-28 17:47:49  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,OGS,2020-12-02 08:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05
1,OGS,2020-12-02 14:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05
2,OGS,2020-12-02 15:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05
3,OGS,2020-12-02 16:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05
4,OGS,2020-12-02 19:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05
...,...,...,...,...,...,...,...
223,OGS,2021-02-23 00:55:00,0.25,-SN,2021-02-23,2021-02-23 06:49:25,2021-02-23 17:41:42
224,OGS,2021-02-25 04:55:00,0.00,-SN,2021-02-25,2021-02-25 06:46:05,2021-02-25 17:44:25
225,OGS,2021-02-25 13:55:00,0.00,-SN,2021-02-25,2021-02-25 06:46:05,2021-02-25 17:44:25
226,OGS,2021-02-25 14:55:00,0.00,-SN,2021-02-25,2021-02-25 06:46:05,2021-02-25 17:44:25


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,OGS,2021-12-01 00:55:00,0.51,-SN,2021-12-01,2021-12-01 07:19:30,2021-12-01 16:22:29
1,OGS,2021-12-01 01:55:00,0.51,-SN,2021-12-01,2021-12-01 07:19:30,2021-12-01 16:22:29
2,OGS,2021-12-01 02:55:00,0.51,-SN,2021-12-01,2021-12-01 07:19:30,2021-12-01 16:22:29
3,OGS,2021-12-01 05:55:00,0.00,-SN,2021-12-01,2021-12-01 07:19:30,2021-12-01 16:22:29
4,OGS,2021-12-01 06:55:00,0.00,-SN,2021-12-01,2021-12-01 07:19:30,2021-12-01 16:22:29
...,...,...,...,...,...,...,...
222,OGS,2022-02-27 01:55:00,0.00,-SN,2022-02-27,2022-02-27 06:43:06,2022-02-27 17:46:48
223,OGS,2022-02-27 02:55:00,0.00,-SN,2022-02-27,2022-02-27 06:43:06,2022-02-27 17:46:48
224,OGS,2022-02-27 13:55:00,0.00,-SN,2022-02-27,2022-02-27 06:43:06,2022-02-27 17:46:48
225,OGS,2022-02-27 14:55:00,0.00,-SN,2022-02-27,2022-02-27 06:43:06,2022-02-27 17:46:48


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,OGS,2022-12-01 13:55:00,0.00,-SN,2022-12-01,2022-12-01 07:19:14,2022-12-01 16:22:34
1,OGS,2022-12-11 14:55:00,0.00,-SN,2022-12-11,2022-12-11 07:29:19,2022-12-11 16:21:05
2,OGS,2022-12-11 15:55:00,0.00,-SN,2022-12-11,2022-12-11 07:29:19,2022-12-11 16:21:05
3,OGS,2022-12-11 16:55:00,0.25,-SN,2022-12-11,2022-12-11 07:29:19,2022-12-11 16:21:05
4,OGS,2022-12-11 17:55:00,0.51,-SN,2022-12-11,2022-12-11 07:29:19,2022-12-11 16:21:05
...,...,...,...,...,...,...,...
263,OGS,2023-02-26 13:55:00,0.00,-SN,2023-02-26,2023-02-26 06:45:12,2023-02-26 17:45:07
264,OGS,2023-02-26 19:55:00,0.00,-SN,2023-02-26,2023-02-26 06:45:12,2023-02-26 17:45:07
265,OGS,2023-02-27 00:55:00,0.00,-SN,2023-02-27,2023-02-27 06:43:31,2023-02-27 17:46:28
266,OGS,2023-02-27 01:55:00,0.00,-SN,2023-02-27,2023-02-27 06:43:31,2023-02-27 17:46:28


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,OGS,2020-12-02 08:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05
1,OGS,2020-12-02 14:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05
2,OGS,2020-12-02 15:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05
7,OGS,2020-12-04 09:55:00,0.51,-SN,2020-12-04,2020-12-04 07:23:02,2020-12-04 16:21:34
8,OGS,2020-12-04 10:55:00,0.51,-SN,2020-12-04,2020-12-04 07:23:02,2020-12-04 16:21:34
...,...,...,...,...,...,...,...
209,OGS,2021-02-19 16:55:00,0.25,-SN,2021-02-19,2021-02-19 06:55:54,2021-02-19 17:36:12
216,OGS,2021-02-22 16:55:00,0.25,-SN,2021-02-22,2021-02-22 06:51:04,2021-02-22 17:40:20
225,OGS,2021-02-25 13:55:00,0.00,-SN,2021-02-25,2021-02-25 06:46:05,2021-02-25 17:44:25
226,OGS,2021-02-25 14:55:00,0.00,-SN,2021-02-25,2021-02-25 06:46:05,2021-02-25 17:44:25


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
0,OGS,2020-12-02 08:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05,2020-12-02 08:20:53,2020-12-02 15:22:05
1,OGS,2020-12-02 14:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05,2020-12-02 08:20:53,2020-12-02 15:22:05
7,OGS,2020-12-04 09:55:00,0.51,-SN,2020-12-04,2020-12-04 07:23:02,2020-12-04 16:21:34,2020-12-04 08:23:02,2020-12-04 15:21:34
8,OGS,2020-12-04 10:55:00,0.51,-SN,2020-12-04,2020-12-04 07:23:02,2020-12-04 16:21:34,2020-12-04 08:23:02,2020-12-04 15:21:34
9,OGS,2020-12-09 09:55:00,0.00,-SN,2020-12-09,2020-12-09 07:27:58,2020-12-09 16:21:02,2020-12-09 08:27:58,2020-12-09 15:21:02
...,...,...,...,...,...,...,...,...,...
207,OGS,2021-02-19 14:55:00,0.00,-SN,2021-02-19,2021-02-19 06:55:54,2021-02-19 17:36:12,2021-02-19 07:55:54,2021-02-19 16:36:12
208,OGS,2021-02-19 15:55:00,0.00,-SN,2021-02-19,2021-02-19 06:55:54,2021-02-19 17:36:12,2021-02-19 07:55:54,2021-02-19 16:36:12
225,OGS,2021-02-25 13:55:00,0.00,-SN,2021-02-25,2021-02-25 06:46:05,2021-02-25 17:44:25,2021-02-25 07:46:05,2021-02-25 16:44:25
226,OGS,2021-02-25 14:55:00,0.00,-SN,2021-02-25,2021-02-25 06:46:05,2021-02-25 17:44:25,2021-02-25 07:46:05,2021-02-25 16:44:25


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.26547169811320753
2021-2022: 0.3261904761904762
2022-2023: 0.27141176470588235


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
3,OGS,2020-12-02 16:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05
4,OGS,2020-12-02 19:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05
5,OGS,2020-12-02 20:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05
6,OGS,2020-12-02 21:55:00,0.00,-SN,2020-12-02,2020-12-02 07:20:53,2020-12-02 16:22:05
16,OGS,2020-12-14 19:55:00,0.00,-SN,2020-12-14,2020-12-14 07:32:09,2020-12-14 16:21:36
...,...,...,...,...,...,...,...
220,OGS,2021-02-22 21:55:00,0.25,-SN,2021-02-22,2021-02-22 06:51:04,2021-02-22 17:40:20
221,OGS,2021-02-22 22:55:00,0.00,-SN,2021-02-22,2021-02-22 06:51:04,2021-02-22 17:40:20
222,OGS,2021-02-22 23:55:00,0.25,-SN,2021-02-22,2021-02-22 06:51:04,2021-02-22 17:40:20
223,OGS,2021-02-23 00:55:00,0.25,-SN,2021-02-23,2021-02-23 06:49:25,2021-02-23 17:41:42


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20032\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.1106
2021-2022: 0.14811320754716983
2022-2023: 0.24000000000000002
